 To create a hello world notebook using nbformat, you can use the following code:


In [ ]:

import nbformat

# Create a new notebook
nb = nbformat.v4.new_notebook()

# Add a code cell to the notebook
code_cell = nbformat.v4.new_code_cell("print('Hello, world!')")
nb.cells.append(code_cell)

# Save the notebook to a file
with open("hello_world.ipynb", "w") as f:
    nbformat.write(nb, f)



To use nbmake's NotebookRun class to execute the notebook from a Python application, you can use the following code:


In [ ]:

from nbmake import NotebookRun

# Create a NotebookRun object
notebook_run = NotebookRun("hello_world.ipynb")

# Execute the notebook
notebook_run.execute()

# Check the output notebook
output_notebook = notebook_run.notebook
assert output_notebook.cells[0].outputs[0].text == "Hello, world!"



Note that the `NotebookRun` class takes a `filename` argument, which is the path to the notebook file to be executed. The `execute` method of the `NotebookRun` object executes the notebook and returns a `NotebookResult` object, which contains information about the execution of the notebook. The `notebook` attribute of the `NotebookResult` object contains the executed notebook.

In the above code, we create a `NotebookRun` object with the path to the notebook file, and then execute the notebook using the `execute` method. We then check the output notebook by accessing the `notebook` attribute of the `NotebookResult` object and checking the output of the first cell.

Note that the `NotebookRun` class also takes a `timeout` argument, which specifies the maximum amount of time to wait for the notebook to execute. If the notebook execution takes longer than the specified timeout, the `NotebookRun` object will raise a `TimeoutError`.